In [1]:
import pandas as pd
import numpy as np
import os
import json

In [2]:
# 대상 폴더에서 데이터프레임 불러오는 반복문 함수
    ## df_1,2,3 이렇게 반복 변수에 순차자정
    
def load_csv_files(folder_path):
    data = {}
    counter = 1
    
    try:
        for file in os.listdir(folder_path):
            if file.endswith(".csv"):
                file_path = os.path.join(folder_path, file)
                df = pd.read_csv(file_path)
                data[f'df_{counter}'] = df
                counter += 1
                
        return data
    except Exception as e:
        print(f"Error: {e}")
        return {}

In [3]:
folder_path = './sample_5'
data = load_csv_files(folder_path)
print(data)

{'df_1':               video_name  width  height  frame_rate  duration  total_frame  \
0  D3_DA_0527_000042.mp4   1920    1080          30     180.0       5400.0   

       film_method filmed_date domain_id   place  ... f1_consis_avg  \
0  filmed_directly  2022-06-22        DA  indoor  ...             1   

   annotated_date version  revision_history           seg_annotator_id  \
0      2022-05-27       1               NaN  [132, 341, 548, 947, 768]   

            seg_confirmer_id distributor          describe_ko  \
0  [515, 300, 679, 730, 148]  selectstar  여자가 물을 마시며 컴퓨터를 한다.   

                             describe_en  \
0  The woman drinks water and computers.   

                                           sentences  
0  [{'timestamps': ['00:00.00000', '00:04.00000']...  

[1 rows x 21 columns], 'df_2':               video_name  width  height  frame_rate  duration  total_frame  \
0  D3_DA_0601_000001.mp4   1920    1080          30     300.0       9000.0   

       film_method film

In [4]:
data['df_3'].head()

,video_name,width,height,frame_rate,duration,total_frame,film_method,filmed_date,domain_id,place,...,f1_consis_avg,annotated_date,version,revision_history,seg_annotator_id,seg_confirmer_id,distributor,describe_ko,describe_en,sentences
0,D3_DA_0603_000001.mp4,1920,1080,30,180.0,5400.0,filmed_directly,2022-06-03,DA,outdoor,...,1,2022-08-02,1,NaN,"[108, 830, 194, 457, 967]","[271, 117, 940, 128, 542]",selectstar,여자가 의자에 누워 스마트폰을 본다.,A woman lies on a chair and looks at her phone.,"[{'timestamps': ['00:00.00000', '00:09.00000']..."


In [5]:
len(data)

5

In [6]:
data.keys()

dict_keys(['df_1', 'df_2', 'df_3', 'df_4', 'df_5'])

In [18]:
data['df_1'].columns

Index(['video_name', 'width', 'height', 'frame_rate', 'duration',
       'total_frame', 'film_method', 'filmed_date', 'domain_id', 'place',
       'f1_consis', 'f1_consis_avg', 'annotated_date', 'version',
       'revision_history', 'seg_annotator_id', 'seg_confirmer_id',
       'distributor', 'describe_ko', 'describe_en', 'sentences'],
      dtype='object')

In [38]:
## 불러온 데이터프레임에서 필요한 컬럼만 추출하는 반복문

df_sel = {}
for key, df in data.items():
    df_sel[key] = df[['video_name', 'describe_en', 'sentences']].copy()
    
for key, df in df_sel.items():
    print(f"{key}:")

df_1:
df_2:
df_3:
df_4:
df_5:


In [39]:
df_sel['df_1']

,video_name,describe_en,sentences
0,D3_DA_0527_000042.mp4,The woman drinks water and computers.,"[{'timestamps': ['00:00.00000', '00:04.00000']..."


In [ ]:
# 전처리 작업
    ## 모든 전처리 코드는 반복문 또는 함수로 생성하기(각각 df에 반복적용 해야 하니까)

In [42]:
df_sel.keys()

dict_keys(['df_1', 'df_2', 'df_3', 'df_4', 'df_5'])

In [43]:
df_sel['df_1']['sentences'][0]

"[{'timestamps': ['00:00.00000', '00:04.00000'], 'sentences_ko': '여자가 냉장고 앞으로 걸어온다.', 'sentences_en': 'The woman walks in front of the refrigerator.'}, {'timestamps': ['00:04.00000', '00:12.00000'], 'sentences_ko': '여자가 냉장고에서 물을 꺼낸다.', 'sentences_en': 'The woman removes the water from the refrigerator.'}, {'timestamps': ['00:12.00000', '00:27.00000'], 'sentences_ko': '여자가 텀블러에 물을 붓는다.', 'sentences_en': 'The woman pours water into the tumbler.'}, {'timestamps': ['00:27.00000', '00:34.00000'], 'sentences_ko': '여자가 텀블러 뚜껑을 닫는다.', 'sentences_en': 'The woman closes the tumbler lid.'}, {'timestamps': ['00:34.00000', '00:40.00000'], 'sentences_ko': '여자가 텀블러와 물을 들고 자리로 이동한다.', 'sentences_en': 'The woman goes to the seat with a tumbler and water.'}, {'timestamps': ['00:40.00000', '00:47.00000'], 'sentences_ko': '여자가 자리에 앉는다.', 'sentences_en': 'The woman sits in place.'}, {'timestamps': ['00:47.00000', '00:52.00000'], 'sentences_ko': '여자가 컴퓨터를 본다.', 'sentences_en': 'The woman sees the computer.'

In [86]:
## 개별 데이터프레임 전처리(문자정규화, 컬럼데이터 추출) 함수

def data_split(df):
    
    new_df = pd.DataFrame(columns=['video_name', 'describe_en', 'data'])
    row_to_add = []
    
    for index, row in df.iterrows():
        sentences_data = eval(row['sentences'])
        for data in sentences_data:
            new_row = {'video_name': row['video_name'],\
                       'describe_en': row['describe_en'], 'data': data}
            row_to_add.append(new_row)
            
    new_df = pd.concat([new_df, pd.DataFrame(row_to_add)], ignore_index=True)
        
    new_df['time_start'] = None
    new_df['time_end'] = None
    new_df['sentences_en'] = None

    for index, row in new_df.iterrows():
        data = row['data']
        new_df.at[index, 'time_start'] = data['timestamps'][0]
        new_df.at[index, 'time_end'] = data['timestamps'][1]
        new_df.at[index, 'sentences_en'] = data['sentences_en']
        
    new_df.drop('data', axis=1, inplace=True)
    new_df['describe_en'] = new_df['describe_en'].str.lower().str.replace('[^a-z0-9\s]', '', regex=True)
    new_df['sentences_en'] = new_df['sentences_en'].str.lower().str.replace('[^a-z0-9\s]', '', regex=True)
    
    return new_df
    

In [91]:
## 전처리 함수 반복실행

key_list = list(df_sel.keys())

df_prep = {}

for key in key_list:
    df_prep[key] = data_split(df_sel[key])
    
for key in df_prep:
    print(f"{key}:")
    print(df_prep[key].head(), "\n")

df_1:
              video_name                           describe_en   time_start  \
0  D3_DA_0527_000042.mp4  the woman drinks water and computers  00:00.00000   
1  D3_DA_0527_000042.mp4  the woman drinks water and computers  00:04.00000   
2  D3_DA_0527_000042.mp4  the woman drinks water and computers  00:12.00000   
3  D3_DA_0527_000042.mp4  the woman drinks water and computers  00:27.00000   
4  D3_DA_0527_000042.mp4  the woman drinks water and computers  00:34.00000   

      time_end                                       sentences_en  
0  00:04.00000       the woman walks in front of the refrigerator  
1  00:12.00000  the woman removes the water from the refrigerator  
2  00:27.00000             the woman pours water into the tumbler  
3  00:34.00000                   the woman closes the tumbler lid  
4  00:40.00000  the woman goes to the seat with a tumbler and ...   

df_2:
              video_name                            describe_en   time_start  \
0  D3_DA_0601_000001.mp

In [92]:
df_prep.keys()

dict_keys(['df_1', 'df_2', 'df_3', 'df_4', 'df_5'])

In [93]:
df_prep['df_1'].head()

,video_name,describe_en,time_start,time_end,sentences_en
0,D3_DA_0527_000042.mp4,the woman drinks water and computers,00:00.00000,00:04.00000,the woman walks in front of the refrigerator
1,D3_DA_0527_000042.mp4,the woman drinks water and computers,00:04.00000,00:12.00000,the woman removes the water from the refrigerator
2,D3_DA_0527_000042.mp4,the woman drinks water and computers,00:12.00000,00:27.00000,the woman pours water into the tumbler
3,D3_DA_0527_000042.mp4,the woman drinks water and computers,00:27.00000,00:34.00000,the woman closes the tumbler lid
4,D3_DA_0527_000042.mp4,the woman drinks water and computers,00:34.00000,00:40.00000,the woman goes to the seat with a tumbler and ...


In [108]:
## 전처리 완료한 데이터프레임 및 메인데이터프레임(각 비디오 파일명을 id 컬럼값으로하는) csv파일 저장

def label_id_to_csv(df_dic, output_dir):
    id_list = []
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for key, df in df_dic.items():
        if not df.empty and 'video_name' in df.columns and len(df['video_name']) > 0:
            file_name = df['video_name'].iloc[0].replace('.mp4', '')
            output_path = os.path.join(output_dir, f'{file_name}.csv')
            df.to_csv(output_path, encoding='utf-8', index=False)
            id_list.append(file_name)
        else:
            print(f"no data or 'video_name' column in dataframe for key: {key}")
    
    df_main = pd.DataFrame({'id': id_list})
    output_main = os.path.join(output_dir, 'df_main.csv')
    df_main.to_csv(output_main, encoding='utf-8', index=False)
    
    for file_name in os.listdir(output_dir):
        print(file_name)
    
    return df_main

In [109]:
output_dir = './preprocessed_label_sample'
main_id = label_id_to_csv(df_prep, output_dir)

D3_DA_0527_000042.csv
D3_DA_0601_000001.csv
D3_DA_0603_000001.csv
D3_DA_0609_000001.csv
D3_DA_0610_000001.csv
df_main.csv


In [110]:
main_id.head()

,id
0,D3_DA_0527_000042
1,D3_DA_0601_000001
2,D3_DA_0603_000001
3,D3_DA_0609_000001
4,D3_DA_0610_000001


In [ ]:
## sentences_en 문장에서 키워드 추출

    ## 단계
        # 토큰화
        # 불용어 제거
        # 품사 태깅
        # 태깅 결과에서 명사, 동사, 형용사만 'desc_kwd' 컬럼에 저장
        

In [8]:
!pip install nltk

     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
     - -------------------------------------- 0.1/1.5 MB 1.7 MB/s eta 0:00:01
     ------------ --------------------------- 0.5/1.5 MB 5.8 MB/s eta 0:00:01
     ----------------------- ---------------- 0.9/1.5 MB 7.0 MB/s eta 0:00:01
     --------------------------------- ------ 1.3/1.5 MB 8.1 MB/s eta 0:00:01
     ---------------------------------------- 1.5/1.5 MB 8.0 MB/s eta 0:00:00
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ---------------------------------------- 42.0/42.0 kB ? eta 0:00:00
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ---------------------------------------- 57.6/57.6 kB ? eta 0:00:00
   ---------------------------------------- 0.0/269.6 kB ? eta -:--:--
   ---------------------------------------- 269.6/269.6 kB 8.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/97.9 kB ? eta -:--:--
   -------------------------

In [9]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\KKM_AI_STUDY90\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\KKM_AI_STUDY90\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\KKM_AI_STUDY90\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [56]:
## basic form test(one csv file apply)

df = pd.read_csv('D3_DA_0527_000042.csv', encoding='utf-8')
df.head(1)

,video_name,describe_en,time_start,time_end,sentences_en
0,D3_DA_0527_000042.mp4,the woman drinks water and computers,00:00.00000,00:04.00000,the woman walks in front of the refrigerator


In [6]:
df['desc_token'] = df['sentences_en'].apply(word_tokenize)

In [11]:
stop_words = set(stopwords.words('english'))

def remove_stopwords(tokens):
    return [word for word in tokens if word not in stop_words]

In [12]:
df['desc_token'] = df['desc_token'].apply(remove_stopwords)

In [13]:
df['desc_token']

0      [woman, walks, front, refrigerator]
1    [woman, removes, water, refrigerator]
2           [woman, pours, water, tumbler]
3            [woman, closes, tumbler, lid]
4      [woman, goes, seat, tumbler, water]
5                     [woman, sits, place]
6                  [woman, sees, computer]
Name: desc_token, dtype: object

In [21]:
def extract_kwd(tokens):
    tagged_tokens = pos_tag(tokens)
    return [word for word, tag in tagged_tokens if tag.startswith('NN') or tag.startswith('VB') or tag.startswith('JJ')]

In [22]:
df['desc_kwd'] = df['desc_token'].apply(extract_kwd)

In [23]:
df['desc_kwd']

0      [woman, walks, front, refrigerator]
1    [woman, removes, water, refrigerator]
2           [woman, pours, water, tumbler]
3            [woman, closes, tumbler, lid]
4      [woman, goes, seat, tumbler, water]
5                     [woman, sits, place]
6                  [woman, sees, computer]
Name: desc_kwd, dtype: object

In [75]:
## 함수로 통합(토큰화, 불용어제거, 품사추출)

import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

stop_words = set(stopwords.words('english'))

def sentence_keywords_extract(df):
    
    column = df['sentences_en']
    processed_column = []   
    tag_list = ['NN', 'VB', 'JJ']
    
    for i in range(len(column)):
        text = column[i]
        tokens = word_tokenize(text)
        
        tokens_no_stopwords = []
        for token in tokens:
            if token not in stop_words:
                tokens_no_stopwords.append(token)
                
        processed_tokens = []
        for word, tag in pos_tag(tokens_no_stopwords):
            if any(tag.startswith(t) for t in tag_list):
                processed_tokens.append(word)
        
        processed_column.append(processed_tokens)
        keywords = processed_tokens
    
    df['sentence_kwd'] = processed_column
    
    return df
    

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\KGM_STC_AI\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\KGM_STC_AI\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\KGM_STC_AI\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [76]:
df_check = sentence_keywords_extract(df)

In [77]:
df_check.head()

,video_name,describe_en,time_start,time_end,sentences_en,sentence_kwd
0,D3_DA_0527_000042.mp4,the woman drinks water and computers,00:00.00000,00:04.00000,the woman walks in front of the refrigerator,"[woman, walks, front, refrigerator]"
1,D3_DA_0527_000042.mp4,the woman drinks water and computers,00:04.00000,00:12.00000,the woman removes the water from the refrigerator,"[woman, removes, water, refrigerator]"
2,D3_DA_0527_000042.mp4,the woman drinks water and computers,00:12.00000,00:27.00000,the woman pours water into the tumbler,"[woman, pours, water, tumbler]"
3,D3_DA_0527_000042.mp4,the woman drinks water and computers,00:27.00000,00:34.00000,the woman closes the tumbler lid,"[woman, closes, tumbler, lid]"
4,D3_DA_0527_000042.mp4,the woman drinks water and computers,00:34.00000,00:40.00000,the woman goes to the seat with a tumbler and ...,"[woman, goes, seat, tumbler, water]"


In [41]:
## 샘플데이터에 일괄 적용

def load_csv_files(folder_path):
    data = {}
    counter = 1
    
    try:
        for file in os.listdir(folder_path):
            if file.endswith(".csv"):
                file_path = os.path.join(folder_path, file)
                df = pd.read_csv(file_path)
                data[f'df_{counter}'] = df
                counter += 1
                
        return data
    except Exception as e:
        print(f"Error: {e}")
        return {}

In [10]:
def extract_columns(df, col_list):
    extracted_df = df[col_list].copy()
    return extracted_df

In [11]:
def data_split(df):
    
    new_df = pd.DataFrame(columns=['video_name', 'describe_en', 'data'])
    row_to_add = []
    
    for index, row in df.iterrows():
        sentences_data = eval(row['sentences'])
        for data in sentences_data:
            new_row = {'video_name': row['video_name'],\
                       'describe_en': row['describe_en'], 'data': data}
            row_to_add.append(new_row)
            
    new_df = pd.concat([new_df, pd.DataFrame(row_to_add)], ignore_index=True)
        
    new_df['time_start'] = None
    new_df['time_end'] = None
    new_df['sentences_en'] = None

    for index, row in new_df.iterrows():
        data = row['data']
        new_df.at[index, 'time_start'] = data['timestamps'][0]
        new_df.at[index, 'time_end'] = data['timestamps'][1]
        new_df.at[index, 'sentences_en'] = data['sentences_en']
        
    new_df.drop('data', axis=1, inplace=True)
    new_df['describe_en'] = new_df['describe_en'].str.lower().str.replace('[^a-z0-9\s]', '', regex=True)
    new_df['sentences_en'] = new_df['sentences_en'].str.lower().str.replace('[^a-z0-9\s]', '', regex=True)
    
    return new_df

In [15]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

stop_words = set(stopwords.words('english'))

def sentence_keywords_extract(df):
    
    column = df['sentences_en']
    processed_column = []   
    tag_list = ['NN', 'VB', 'JJ']
    
    for i in range(len(column)):
        text = column[i]
        tokens = word_tokenize(text)
        
        tokens_no_stopwords = []
        for token in tokens:
            if token not in stop_words:
                tokens_no_stopwords.append(token)
                
        processed_tokens = []
        for word, tag in pos_tag(tokens_no_stopwords):
            if any(tag.startswith(t) for t in tag_list):
                processed_tokens.append(word)
        
        processed_column.append(processed_tokens)
    
    df['sentence_kwd'] = processed_column
    
    return df

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\KKM_AI_STUDY90\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\KKM_AI_STUDY90\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\KKM_AI_STUDY90\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [20]:
def label_id_to_csv(df_dic, output_dir):
    id_list = []
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for key, df in df_dic.items():
        if not df.empty and 'video_name' in df.columns and len(df['video_name']) > 0:
            file_name = df['video_name'].iloc[0].replace('.mp4', '')
            output_path = os.path.join(output_dir, f'{file_name}.csv')
            df.to_csv(output_path, encoding='utf-8', index=False)
            id_list.append(file_name)
        else:
            print(f"no data or 'video_name' column in dataframe for key: {key}")
    
    df_main = pd.DataFrame({'id': id_list})
    output_main = os.path.join(output_dir, 'df_main.csv')
    df_main.to_csv(output_main, encoding='utf-8', index=False)
    
    for file_name in os.listdir(output_dir):
        print(file_name)
    
    return df_main

In [16]:
## 전처리 일괄
    ## 데이터 로드: load_csv_files(folder_path)
    ## 컬럼 추출: extract_columns(df, col_list)
    ## 정규화 및 컬럼데이터 분리:data_split(df)
    ## 키워드 추출: sentences_keywords_extract(column)
    ## csv 파일 생성(label, main): label_id_to_csv(df_dic, output_dir)

## 데이터 로드
folder_path = './sample_5'
data = load_csv_files(folder_path)

## 데이터프레임 전처리
### 컬럼 추출
col_list = ['video_name', 'describe_en', 'sentences']

df_sel = {}
for key, df in data.items():
    df_sel[key] = extract_columns(df, col_list)
    
### 정규화 및 컬럼데이터 분리
df_split = {}
for key, df in df_sel.items():
    df_split[key] = data_split(df)
    
### 키워드 추출
df_prep = {}
for key, df in df_split.items():
    df_prep[key] = sentence_keywords_extract(df)

In [17]:
## 결과 확인
print(df_prep.keys())
a = len(df_prep['df_1'])
b = len(df_prep['df_2'])
c = len(df_prep['df_3'])
d = len(df_prep['df_4'])
e = len(df_prep['df_5'])
print(a+b+c+d+e)



dict_keys(['df_1', 'df_2', 'df_3', 'df_4', 'df_5'])
31


In [23]:
## 전처리 데이터프레임 csv 파일 저장
output_dir = './sample_label_preprocessed'
main_id = label_id_to_csv(df_prep, output_dir)

D3_DA_0527_000042.csv
D3_DA_0601_000001.csv
D3_DA_0603_000001.csv
D3_DA_0609_000001.csv
D3_DA_0610_000001.csv
df_main.csv


In [24]:
main_df = pd.read_csv('./sample_label_preprocessed/df_main.csv', encoding='utf-8')

In [25]:
main_df.head()

,id
0,D3_DA_0527_000042
1,D3_DA_0601_000001
2,D3_DA_0603_000001
3,D3_DA_0609_000001
4,D3_DA_0610_000001
